# **MACHINE LEARNING FOR MALWARE DETECTION IN ANDROID**

#Introduction


Done by Ananiya Anantharaman, Siddhi Aadekar, Raj Shedge

## **Training Section**

In [ ]:
# Install required packages
!pip install --upgrade androguard pyaxmlparser opendatasets

# Import libraries
from pyaxmlparser import APK
import pandas as pd
import pickle
import opendatasets as od
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Download the dataset from Kaggle
od.download('https://www.kaggle.com/datasets/ankit1743/android-malware-detection-dataset')

# Load the dataset
data = pd.read_csv('/content/android-malware-detection-dataset/Android_Malware.csv')

# Prepare features and labels
X = data.drop('Result', axis=1)
y = data['Result']
PERMISSION_LIST = list(X.columns)

# Split dataset into training and testing sets for proper evaluation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the Random Forest model on training data
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Save the trained model
with open('rf_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Load the model
model = pickle.load(open('rf_model.pkl', 'rb'))

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 964.5/964.5 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 138.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.5/32.5 MB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.4/247.4 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.6/607.6 kB 44.0 MB/s eta 0:00:00
  Created wheel for mutf8: filename=mutf8-1.0.6-cp312-cp312-linux_x86_6

100%|██████████| 250k/250k [00:00<00:00, 773MB/s]

#**Evaluation Section**

In [ ]:
# Predict on test data
y_pred = model.predict(X_test)

# Evaluate metrics
print("=== Model Evaluation on Test Data ===")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-score:", f1_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nDetailed Classification Report:\n", classification_report(y_test, y_pred))



=== Model Evaluation on Test Data ===
Accuracy: 0.97125
Precision: 0.9764542936288089
Recall: 0.9655329833371377
F1-score: 0.9709629289567313
Confusion Matrix:
 [[4317  102]
 [ 151 4230]]

Detailed Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.98      0.97      4419
           1       0.98      0.97      0.97      4381

    accuracy                           0.97      8800
   macro avg       0.97      0.97      0.97      8800
weighted avg       0.97      0.97      0.97      8800



#**Testing Section**

In [ ]:
# Function to extract permissions from APK and build feature vector
def extract_permissions_pyaxml(apk_path):
    apk = APK(apk_path)
    perms = apk.get_permissions()
    perms_vector = [1 if perm in perms else 0 for perm in PERMISSION_LIST]
    return perms_vector

# Updated scan function with adjustable malware probability threshold
def scan_apk_pyaxml(apk_path, malware_probability_threshold=0.2):
    try:
        features = extract_permissions_pyaxml(apk_path)
        features_df = pd.DataFrame([features], columns=PERMISSION_LIST)
        prob = model.predict_proba(features_df)[0]
        print("Probability (Benign, Malware):", prob)
        if prob[1] >= malware_probability_threshold:
            print(f"Malware detected! Malware probability {prob[1]:.2f} ≥ threshold {malware_probability_threshold}")
        else:
            print(f"App is benign. Malware probability {prob[1]:.2f} < threshold {malware_probability_threshold}")
    except Exception as e:
        print("Error scanning APK:", e)

# For uploading APK file in Colab, run this in the notebook:
from google.colab import files
uploaded = files.upload()  # Upload your APK file here

# Example scan call (update filename accordingly)
scan_apk_pyaxml("/content/wildfire-test-apk-file.apk", malware_probability_threshold=0.2)

# Evaluate on test data
print("Test accuracy:", model.score(X_test, y_test))

KeyboardInterrupt: 

In [ ]:
scan_apk_pyaxml("/content/testmalware.apk", malware_probability_threshold=0.2)

Probability (Benign, Malware): [0.78907362 0.21092638]
Malware detected! Malware probability 0.21 ≥ threshold 0.2


#Conclusion


##Thank You!